# TV

In [4]:
import numpy as np
import pandas as pd
import pandas_ta as ta
from datetime import datetime,timedelta
import plotly.express as px

In [5]:
import vectorbtpro as vbt
vbt.settings.set_theme("dark")

In [6]:
data = vbt.TVData.fetch(
    'NASDAQ:GOOG',
    timeframe = '1 day',
)
print(type(data.get()))

data.run("rsi",14).rsi
# vbt.EMA.run(close = data['Close'],length=14)
# data
# data.index = pd.to_datetime(data.index.date)
# start_date = '2017-09-27' 
# end_date = '2023-03-31'
# mask  = (data.index > start_date) & (data.index <= end_date)
# # data = data.loc[mask]
# data = data.round(2)
# data
len(data.get().loc[:,"Close"])
len(data.get()["Close"])

<class 'pandas.core.frame.DataFrame'>


2275

In [7]:
df = pd.DataFrame(data.get())
# fast_ema = data.run("ema", 10, hide_params=True)
ema = vbt.IF.from_pandas_ta('EMA')
# ema1 = EMA.run(df['Close'], length = 10).ema
# fast_ema.real_crossed_above(20)
a1 = ema.run(df['Close'], length = 10).ema
a2 = ema.run(df['Close'], length = 20).ema
a2.vbt.crossed_above(a1)
# df['Entry'] = df[df['EMA35'] > df['EMA50']]
# ema2 = vbt.IF.from_pandas_ta('EMA')
# ema.run(df['Close'], length = 20).ema
# fast_ema, slow_ema  = vbt.ema.run(close=df['Close'], length = 35).ema, vbt.ema.run(close=df['Close'], length = 50).ema
# entries = fast_ema.vbt.crossed_above(slow_ema)


datetime
2014-03-27 13:30:00+00:00    False
2014-03-28 13:30:00+00:00    False
2014-03-31 13:30:00+00:00    False
2014-04-01 13:30:00+00:00    False
2014-04-02 13:30:00+00:00    False
                             ...  
2023-04-03 13:30:00+00:00    False
2023-04-04 13:30:00+00:00    False
2023-04-05 13:30:00+00:00    False
2023-04-06 13:30:00+00:00    False
2023-04-10 13:30:00+00:00    False
Length: 2275, dtype: bool

In [8]:
# pf = vbt.PF.from_random_signals(
#     data, 
#     n=1,
#     tp_stop=0.5, 
#     sl_stop=0.1
# )
# pf.trade_history

# Check Indicator

In [10]:
ind = pd.DataFrame()
print(ind.ta.indicators())

Pandas TA - Technical Analysis Indicators - v0.3.14b0
Total Indicators & Utilities: 205
Abbreviations:
    aberration, above, above_value, accbands, ad, adosc, adx, alma, amat, ao, aobv, apo, aroon, atr, bbands, below, below_value, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chop, cksp, cmf, cmo, coppock, cross, cross_value, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, fisher, fwma, ha, hilo, hl2, hlc3, hma, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, rma, roc, rsi, rsx, rvgi, rvi, short_run, sinwma, skew, slope, sma, smi, squeeze, squeeze_pro, ssf, stc, stdev, stoch, stochrsi, supertrend, swma, t3, td_seq, tema, thermo, tos_stdevall, trima, trix, true_range, tsi, tsignals, ttm_trend, ui, 

# EMA Function


In [ ]:
# data, , startdate, enddate, , , , , ,, , , , , , , , , , equal,length
def EMA_CrossOver(symbol = None, timeframe = '1 day', sl_stop = None, tp_stop = None, tsl_stop = None, size = 1, max_size = None, init_cash = 100,
                  fast_length = 15, slow_length = 50, start_date = None, end_date = None):
    ''' 
    symbol = '{Market}:{Stock}'
    start_date = '{Year}-{Month}-{Day}' 
    end_date = '{Year}-{Month}-{Day}'
    slow_length = slow_ema
    fast_length = fast_ema
    tp_stop, sl_stop, tsl_stop = percentage / 100
    '''
    data = vbt.TVData.fetch(symbol, timeframe = timeframe)    
    #initialize dataFrame
    df = pd.DataFrame(data.get())
    df = df.round(2)
    df.index = pd.to_datetime(df.index.date)
   
    # EMA
    ema = vbt.IF.from_pandas_ta('EMA')
    fast_ema, slow_ema  = ema.run(close = df['Close'], length = fast_length).ema, ema.run(close = df['Close'], length = slow_length).ema
    entries = fast_ema.vbt.crossed_above(slow_ema)
    exits = fast_ema.vbt.crossed_below(slow_ema)

    # Datetime
    if(start_date is not None and end_date is not None):
        mask  = (df.index >= start_date) & (df.index <= end_date)
        df = df.loc[mask]

    pf = vbt.Portfolio.from_signals(open = df["Open"], close = df["Close"], high = df["High"], low = df["Low"], 
                                    entries = entries, exits = exits, sl_stop = sl_stop, tp_stop = tp_stop, tsl_stop = tsl_stop,
                                    size = size, min_size = 1, max_size = max_size, init_cash = init_cash, price = 'NextOpen', stop_entry_price = 'FillPrice')
    return pf

In [ ]:
# EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, start_date = '2017-09-27' ,end_date = '2023-02-25', tp_stop = 0.1).stats(settings=dict(freq='d'))
pf = EMA_CrossOver(symbol = 'NASDAQ:GOOG', init_cash = 1000, start_date = '2017-09-27' ,end_date = '2023-02-25', tp_stop = 0.1)
# pf.orders.records_readable
# pf[ df.index == '2017-09-29']
# pf.orders.records_readable
#pf = pd.DataFrame(pf.orders.records_readable)

pf.stats()
# help(EMA_CrossOver)
pf.plot().show()
# pf.trade_history

/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/generic/stats_builder.py:463: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/Users/jib/opt/anaconda3/envs/vectorbt/lib/python3.8/site-packages/vectorbtpro/base/wrapping.py:961: UserWarning: Couldn't parse the frequency of index. Pass it as `freq` or de

,Order Id,Column,Signal Index,Creation Index,Fill Index,Side,Type,Stop Type,Size,Price,Fees,PnL,Return,Direction,Status,Entry Trade Id,Exit Trade Id,Position Id
0,0,0,2017-09-28,2017-09-29,2017-09-29,Buy,Market,None,1.0,47.600,0.0,4.760,0.100000,Long,Closed,0,-1,0
1,1,0,2017-09-29,2017-10-27,2017-10-27,Sell,Market,TP,1.0,52.360,0.0,4.760,0.100000,Long,Closed,-1,0,0
2,2,0,2018-05-14,2018-05-15,2018-05-15,Buy,Market,None,1.0,54.500,0.0,5.450,0.100000,Long,Closed,1,-1,1
3,3,0,2018-05-15,2018-07-17,2018-07-17,Sell,Market,TP,1.0,59.950,0.0,5.450,0.100000,Long,Closed,-1,1,1
4,4,0,2018-10-03,2018-10-04,2018-10-04,Buy,Market,None,1.0,59.770,0.0,-1.390,-0.023256,Long,Closed,2,-1,2
5,5,0,2018-10-04,2018-10-05,2018-10-05,Sell,Market,None,1.0,58.380,0.0,-1.390,-0.023256,Long,Closed,-1,2,2
6,6,0,2019-01-18,2019-01-22,2019-01-22,Buy,Market,None,1.0,54.400,0.0,5.440,0.100000,Long,Closed,3,-1,3
7,7,0,2019-01-22,2019-03-12,2019-03-12,Sell,Market,TP,1.0,59.840,0.0,5.440,0.100000,Long,Closed,-1,3,3
8,8,0,2019-07-16,2019-07-17,2019-07-17,Buy,Market,None,1.0,57.550,0.0,5.755,0.100000,Long,Closed,4,-1,4
9,9,0,2019-07-17,2019-10-25,2019-10-25,Sell,Market,TP,1.0,63.305,0.0,5.755,0.100000,Long,Closed,-1,4,4
